In [2]:
import os
import numpy as np
import pandas as pd
import statistics
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json
from pandas import json_normalize
path = os.getcwd()

In [3]:
def add_br(s):
    if 'ScCCreal' in s and 'ScCCcomb' not in s and 'ScACOcomb' not in s and 'ScGAcomb' not in s:
        if 'ScCCreal' == s:
            return 'ScCCreal'
        res_list = s.split('ScCCreal')
        return 'ScCCreal' + "<br>" + res_list[1]
    if 'ScCCcomb' in s:
        if 'ScCCcomb' == s:
            return 'ScCCcomb'
        res_list = s.split('ScCCcomb')
        return 'ScCCcomb' + "<br>" + res_list[1]
    if '(best)' in s:
        res_list = s.split('(best)')
        return res_list[0] + "<br>" + 'best'
    if '(average)' in s:
        res_list = s.split('(average)')
        return res_list[0]
    if 'ScACOcomb' in s:
        res_list = s.split('ScACOcomb')
        return 'ScACOcomb' + "<br>" + res_list[1]
    if 'ScGAcomb' in s:
        res_list = s.split('ScGAcomb')
        return 'ScGAcomb' + "<br>" + res_list[1]
    return s

In [4]:
data = pd.read_excel(path + "\\" + 'Scheduling_results_simple.xlsx', sheet_name=None)
data=data['All']
data_copy = data.copy()
data.head()

,Problem,Algorithm,Problem 0,Problem 1,Problem 2,Problem 3,Problem 4,Problem 5
0,LO,LKH,5044.14,7937.86,16500.00,10775.80,48190.06,30375.42
1,LO,GA (best),5025.00,7798.98,16445.44,10279.46,46716.50,29979.60
2,LO,GA (average),5026.77,7885.88,16455.84,10542.65,47369.18,30136.99
3,LO,ScGA,5025.00,7854.89,16451.17,10363.71,46867.94,30017.29
4,LO,IWDs (best),5316.90,8281.50,16782.78,11152.64,48775.06,30630.66


In [5]:
problems = ['Problem 0', 'Problem 1', 'Problem 2', 'Problem 3', 'Problem 4', 'Problem 5']
for problem in problems:
    m = max(data[problem])
    data[problem] = data[problem] / m
data["Problem avg"] = (data['Problem 0']+data['Problem 1']+data['Problem 2']+data['Problem 3']+data['Problem 4']+data['Problem 5']) / 6.0
data["Problem avg"] = round(data["Problem avg"], 3)
data["Algorithm br"] = data["Algorithm"].apply(lambda x: add_br(x))
data.head()

,Problem,Algorithm,Problem 0,Problem 1,Problem 2,Problem 3,Problem 4,Problem 5,Problem avg,Algorithm br
0,LO,LKH,0.921307,0.947140,0.976450,0.935028,0.983618,0.982250,0.958,LKH
1,LO,GA (best),0.917812,0.930569,0.973221,0.891960,0.953541,0.969451,0.939,GA <br>best
2,LO,GA (average),0.918135,0.940938,0.973836,0.914798,0.966863,0.974540,0.948,GA
3,LO,ScGA,0.917812,0.937240,0.973560,0.899271,0.956632,0.970669,0.943,ScGA
4,LO,IWDs (best),0.971127,0.988143,0.993184,0.967727,0.995558,0.990504,0.984,IWDs <br>best


In [10]:
patterns = ('', '/', '\\', 'x', '-', '|', '+', '.', '', '/', '\\', 'x', '-', '|', '+', '.', 
            '', '/', '\\', 'x', '-', '|', '+', '.')
templates = ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]

In [99]:
def apply_style(fig, width=700, height=400, range=[0.8, 1]):
    fig.update_xaxes(tickangle=90)
    fig.update_layout(autosize=False, width=width, height=height, showlegend=False, 
        font=dict(family="Times New Roman", size=20, color='black'),
                 template="plotly_white"
    )
    fig.update_xaxes(title='', showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(title='', showline=True, linewidth=1, linecolor='black', gridcolor='lightgray')
    fig.update_traces(marker_pattern_size=6, marker_line_color='black')
    fig.update_yaxes(range=range)
    fig.update_annotations(font_size=20)

In [100]:
def make_bar_plot(data, width=700, height=400, color=True, range=[0.8, 1]):
    if color:
        color_discrete_sequence = px.colors.sequential.Viridis
    else:
        color_discrete_sequence = px.colors.sequential.Greys
    fig = px.bar(data, x='Algorithm br', y='Problem avg', color='Algorithm br', labels={'value':'reliability'}, 
             color_discrete_sequence=color_discrete_sequence, text_auto=True,
             pattern_shape="Algorithm br", pattern_shape_sequence=patterns)
    apply_style(fig, width, height, range)
    return fig

In [68]:
algorithms_LO = ['GA (average)', 'ScGA', 'GA (best)', 'ACO (average)', 'ScACO', 'ACO (best)', 'IWDs (average)', 
                 'IWDs (best)', 'LKH', 'ScCCcomb', 'ScCCcomb SBSP', 'ScCCcomb BtA BrW','ScCCcomb BtA BrR', 
                 'ScCCcomb BtA RrW', 'ScCCcomb BtA RrR', 
                 'ScCCcomb AtA BrW', 'ScCCcomb AtA BrR', 'ScCCcomb AtA RrW', 'ScCCcomb AtA RrR']

In [69]:
algorithms_LO = ['GA (average)', 'ScGA', 'GA (best)', 'ACO (average)', 'ScACO', 'ACO (best)', 'IWDs (average)', 
                 'IWDs (best)', 'LKH', 'ScCCcomb']
data_LO = data[data['Problem'] == 'LO'].copy()
data_LO = data_LO[data_LO['Algorithm'].isin(algorithms_LO)]
data_LO.sort_values(by="Algorithm", key=lambda column: column.map(lambda e: algorithms_LO.index(e)), inplace=True)
data_LO = data_LO.reset_index()
data_LO = data_LO.reset_index()

In [85]:
make_bar_plot(data_LO)

In [86]:
make_bar_plot(data_LO, False)

In [106]:
def make_bar_plot_for_problems(data, width=1200, height=1200, range=[0.8, 1]):
    fig = make_subplots(rows=3, cols=2, subplot_titles=problems)
    count_x = 0
    couny_y = 0
    count = 0

    for problem in problems:
        if count % 2 == 1:
            count_y = 2
        else:
            count_y = 1
            count_x += 1
        data[problem] = data[problem].round(decimals = 3)
        fig.add_trace(go.Bar(x=data['Algorithm br'], y=data[problem], text=data[problem],
                             marker=dict(color = data['level_0'], colorscale='viridis')), 
                      count_x, count_y)
        count = count + 1

    fig.update_traces(marker_pattern_shape=patterns)
    apply_style(fig, width, height, range)
    return fig

In [107]:
make_bar_plot_for_problems(data_LO).show()

In [78]:
algorithms_PO = ['GA (average)', 'ScGA', 'GA (best)', 'ACO (average)', 'ScACO', 'ACO (best)', 'IWDs (average)', 
                 'IWDs (best)', 'LKH', 'ScCCcomb']
data_PO = data[data['Problem'] == 'PO'].copy()
data_PO = data_PO[data_PO['Algorithm'].isin(algorithms_PO)]
data_PO.sort_values(by="Algorithm", key=lambda column: column.map(lambda e: algorithms_PO.index(e)), inplace=True)
data_PO = data_PO.reset_index()
data_PO = data_PO.reset_index()
data_PO.head()

,level_0,index,Problem,Algorithm,Problem 0,Problem 1,Problem 2,Problem 3,Problem 4,Problem 5,Problem avg,Algorithm br
0,0,21,PO,GA (average),0.857,0.915684,0.949061,0.905035,0.920612,0.967017,0.919,GA
1,1,22,PO,ScGA,0.843,0.902677,0.946328,0.883844,0.900798,0.960310,0.906,ScGA
2,2,20,PO,GA (best),0.834,0.901689,0.946347,0.904303,0.920244,0.966705,0.912,GA <br>best
3,3,26,PO,ACO (average),0.894,0.929829,0.955706,0.907102,0.916313,0.967115,0.928,ACO
4,4,27,PO,ScACO,0.885,0.930894,0.955690,0.896554,0.905882,0.963119,0.923,ScACO


In [88]:
make_bar_plot(data_PO)

In [89]:
make_bar_plot(data_PO, False)

In [104]:
make_bar_plot_for_problems(data_PO)

In [81]:
algorithms_AP = ['PSO (average)', 'ScPSO', 'PSO (best)', 'RGA (average)', 'ScRGA', 'RGA (best)', 
                 'DE (average)', 'ScDE', 'DE (best)', 'ScCCreal']
data_AP = data[data['Problem'] == 'AP'].copy()
data_AP = data_AP[data_AP['Algorithm'].isin(algorithms_AP)]
data_AP.sort_values(by="Algorithm", key=lambda column: column.map(lambda e: algorithms_AP.index(e)), inplace=True)
data_AP = data_AP.reset_index()
data_AP = data_AP.reset_index()

In [90]:
make_bar_plot(data_AP)

In [91]:
make_bar_plot(data_AP, False)

In [103]:
make_bar_plot_for_problems(data_AP, [0.75, 1])

In [97]:
algorithms_LO_MTP = ['ScCCcomb ScCCreal', 'ScCCcomb ScPSOreal', 'ScCCcomb ScRGAreal', 'ScCCcomb ScDEreal', 
                 'ScACOcomb ScCCreal', 'ScACOcomb ScPSOreal', 'ScACOcomb ScRGAreal', 'ScACOcomb ScDEreal', 
                 'ScGAcomb ScCCreal', 'ScGAcomb ScPSOreal', 'ScGAcomb ScRGAreal', 'ScGAcomb ScDEreal']
data_LO_MTP = data[data['Problem'] == 'LO-MTP'].copy()
data_LO_MTP.sort_values(by="Algorithm", key=lambda column: column.map(lambda e: algorithms_LO_MTP.index(e)), inplace=True)
data_LO_MTP = data_LO_MTP.reset_index()
data_LO_MTP = data_LO_MTP.reset_index()

In [101]:
make_bar_plot(data_LO_MTP, width=900, height=500)

In [102]:
make_bar_plot(data_LO_MTP, width=900, height=500, color=False)

In [108]:
make_bar_plot_for_problems(data_LO_MTP, width=1400, height=1400)

In [109]:
def niormalize(data_sccc):
    for problem in problems:
        m = max(data_sccc[problem])
        data_sccc[problem] = data_sccc[problem] / m
    data_sccc["Problem avg"] = (data_sccc['Problem 0']+data_sccc['Problem 1']+data_sccc['Problem 2']+data_sccc['Problem 3']+data_sccc['Problem 4']+data_sccc['Problem 5']) / 6.0
    data_sccc["Problem avg"] = round(data_sccc["Problem avg"], 3)
    data_sccc["Algorithm br"] = data_sccc["Algorithm"].apply(lambda x: add_br(x))
    return data_sccc

In [110]:
sccc_algorithms = ['ScCCcomb SBSP', 'ScCCcomb BtA BrW','ScCCcomb BtA BrR', 'ScCCcomb BtA RrW', 'ScCCcomb BtA RrR',
                   'ScCCcomb AtA BrW', 'ScCCcomb AtA BrR', 'ScCCcomb AtA RrW', 'ScCCcomb AtA RrR', 
                   'ScCCreal SBSP', 'ScCCreal BtA BrW', 'ScCCreal BtA BrR', 'ScCCreal BtA RrW', 'ScCCreal BtA RrR', 
                   'ScCCreal AtA BrW', 'ScCCreal AtA BrR', 'ScCCreal AtA RrW', 'ScCCreal AtA RrR']
data_sccc = data_copy[data_copy['Algorithm'].isin(sccc_algorithms)]
data_sccc.head()

,Problem,Algorithm,Problem 0,Problem 1,Problem 2,Problem 3,Problem 4,Problem 5
10,LO,ScCCcomb SBSP,5025.0,7898.5,16454.0,10423.7,47030.2,30056.1
11,LO,ScCCcomb BtA BrW,5025.0,7904.1,16451.7,10421.7,46961.2,30048.7
12,LO,ScCCcomb BtA BrR,5025.0,7907.8,16454.0,10407.2,46970.0,30061.1
13,LO,ScCCcomb BtA RrW,5025.0,7904.3,16454.0,10446.1,47047.9,30031.9
14,LO,ScCCcomb BtA RrR,5025.0,7926.0,16454.0,10400.6,47049.4,30038.0


In [114]:
data_LO_sccc = data_sccc[data_sccc['Problem'] == 'LO'].copy()
data_LO_sccc = niormalize(data_LO_sccc)
data_LO_sccc = data_LO_sccc.reset_index()
data_LO_sccc = data_LO_sccc.reset_index()

In [121]:
make_bar_plot(data_LO_sccc, width=700, height=400)

In [120]:
make_bar_plot_for_problems(data_LO_sccc, range=[0.95, 1])

In [122]:
data_PO_sccc = data_sccc[data_sccc['Problem'] == 'PO'].copy()
data_PO_sccc = niormalize(data_PO_sccc)
data_PO_sccc = data_PO_sccc.reset_index()
data_PO_sccc = data_PO_sccc.reset_index()

In [123]:
make_bar_plot(data_PO_sccc, width=700, height=400)

In [124]:
make_bar_plot_for_problems(data_PO_sccc, range=[0.95, 1])

In [130]:
algorithms_LO_short = ['GA (average)', 'ScGA', 'ACO (average)', 'ScACO', 'IWDs (average)', 'LKH', 'ScCCcomb']
algorithms_PO_short = ['GA (average)', 'ScGA', 'ACO (average)', 'ScACO', 'IWDs (average)', 'LKH', 'ScCCcomb']
algorithms_AP_short = ['PSO (average)', 'ScPSO', 'RGA (average)', 'ScRGA', 'DE (average)', 'ScDE', 'ScCCreal']
algorithms_LO_MTP_short = ['ScCCcomb ScCCreal']

In [137]:
def concat_data(df_output, data, problem_name, algorithms):
    data_short = data[data['Algorithm'].isin(algorithms)]
    ser = data_short[problems].mean(axis=0)
    ser['Problem name'] = problem_name
    return pd.concat([df_output, ser.to_frame().T],ignore_index=True)

In [140]:
columns = [x for x in problems]
columns.append('Problem name')
df_prob_stat = pd.DataFrame(columns = columns)
df_prob_stat = concat_data(df_prob_stat, data_LO, 'LO', algorithms_LO_short)
df_prob_stat = concat_data(df_prob_stat, data_PO, 'PO', algorithms_PO_short)
df_prob_stat = concat_data(df_prob_stat, data_AP, 'AP', algorithms_AP_short)
df_prob_stat = concat_data(df_prob_stat, data_LO_MTP, 'LO-MTP', algorithms_LO_MTP_short)
df_prob_stat = df_prob_stat.set_index('Problem name')
df = df_prob_stat.T

In [146]:
def apply_style_scatter(fig, width=700, height=400, range=[0.8, 1]):    
    fig.update_layout(autosize=False, width=width, height=height, showlegend=True,
                      template="plotly_white", 
                      font=dict(family="Times New Roman", size=18, color='black')
    )
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='lightgray')
    fig.update_traces(marker_line_color='black')
    fig.update_yaxes(range=range)
    fig.update_annotations(font_size=20)

In [149]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=df['LO'], mode = 'lines+markers', name="LO", 
                          line=dict(color='#440154', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=df['PO'], mode = 'lines+markers', name="PO", 
                          line=dict(color='#26828e', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=df['AP'], mode = 'lines+markers', name="AP", 
                          line=dict(color='#6ece58', width=width, dash='dash'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=df['LO-MTP'], mode = 'lines+markers', name="LO-MTP", 
                          line=dict(color='#fde725', width=width, dash='dash'),
                          marker=dict(symbol="circle", size=7)))

apply_style_scatter(fig, range=[0.85, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")
fig.show()

In [150]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=df['LO'], mode = 'lines+markers', name="LO", 
                          line=dict(color='#ced4da', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=df['PO'], mode = 'lines+markers', name="PO", 
                          line=dict(color='#adb5bd', width=width, dash='longdash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=df['AP'], mode = 'lines+markers', name="AP", 
                          line=dict(color='#6c757d', width=width, dash='dot'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=df['LO-MTP'], mode = 'lines+markers', name="LO-MTP", 
                          line=dict(color='black', width=width, dash='dash'),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig, range=[0.85, 1])
fig.show()

In [151]:
df_prob_stat_sccc = pd.DataFrame(columns = columns)
df_prob_stat_sccc = concat_data(df_prob_stat_sccc, data_LO, 'LO', ['ScCCcomb'])
df_prob_stat_sccc = concat_data(df_prob_stat_sccc, data_PO, 'PO', ['ScCCcomb'])
df_prob_stat_sccc = concat_data(df_prob_stat_sccc, data_AP, 'AP', ['ScCCreal'])

ser = data_LO_MTP[problems].mean(axis=0)
ser['Problem name'] = 'LO-MTP'
df_prob_stat_sccc = pd.concat([df_prob_stat_sccc, ser.to_frame().T],ignore_index=True)

df_prob_stat_sccc = df_prob_stat_sccc.set_index('Problem name')
df_sccc = df_prob_stat_sccc.T

In [154]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=df_sccc['LO'], mode = 'lines+markers', name="LO", 
                          line=dict(color='#440154', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=df_sccc['PO'], mode = 'lines+markers', name="PO", 
                          line=dict(color='#26828e', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=df_sccc['AP'], mode = 'lines+markers', name="AP", 
                          line=dict(color='#6ece58', width=width, dash='dash'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=df_sccc['LO-MTP'], mode = 'lines+markers', name="LO-MTP", 
                          line=dict(color='#fde725', width=width, dash='dash'),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig)
fig.show()

In [155]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=df_sccc['LO'], mode = 'lines+markers', name="LO", 
                          line=dict(color='#ced4da', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=df_sccc['PO'], mode = 'lines+markers', name="PO", 
                          line=dict(color='#adb5bd', width=width, dash='longdash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=df_sccc['AP'], mode = 'lines+markers', name="AP", 
                          line=dict(color='#6c757d', width=width, dash='dot'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=df_sccc['LO-MTP'], mode = 'lines+markers', name="LO-MTP", 
                          line=dict(color='black', width=width, dash='dash'),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig)
fig.show()

In [156]:
cols = ['Algorithm', 'Problem 0', 'Problem 1', 'Problem 2', 'Problem 3', 'Problem 4', 'Problem 5']

In [157]:
data_LO_i = data_LO[data_LO['Algorithm'].isin(['GA (average)', 'ScGA', 'ACO (average)', 'ScACO', 'IWDs (average)', 
                                               'LKH', 'ScCCcomb'])][cols]
data_LO_i = data_LO_i.set_index(['Algorithm'])
data_LO_i = data_LO_i.transpose()

In [162]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=data_LO_i['GA (average)'],
                          mode = 'lines+markers', name="GA", 
                          line=dict(color='#440154', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['ScGA'],
                          mode = 'lines+markers', name="ScGA", 
                          line=dict(color='#440154', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['ACO (average)'],
                          mode = 'lines+markers', name="ACO", 
                          line=dict(color='#26828e', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_i['ScACO'],
                          mode = 'lines+markers', name="ScACO", 
                          line=dict(color='#26828e', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_i['IWDs (average)'],
                          mode = 'lines+markers', name="IWDs", 
                          line=dict(color='#6ece58', width=width),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['LKH'],
                          mode = 'lines+markers', name="LKH", 
                          line=dict(color='#6ece58', width=width, dash='dash'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['ScCCcomb'], mode = 'lines+markers', name="ScCCcomb", 
                          line=dict(color='#fde725', width=width),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig, range=[0.88, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")
fig.show()

In [168]:
fig = go.Figure()
width=2
fig.add_traces(go.Scatter(y=data_LO_i['GA (average)'],
                          mode = 'lines+markers', name="GA", 
                          line=dict(color='#ced4da', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['ScGA'],
                          mode = 'lines+markers', name="ScGA", 
                          line=dict(color='#ced4da', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['ACO (average)'],
                          mode = 'lines+markers', name="ACO", 
                          line=dict(color='#adb5bd', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_i['ScACO'],
                          mode = 'lines+markers', name="ScACO", 
                          line=dict(color='#adb5bd', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_i['IWDs (average)'],
                          mode = 'lines+markers', name="IWDs", 
                          line=dict(color='#6c757d', width=width),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['LKH'],
                          mode = 'lines+markers', name="LKH", 
                          line=dict(color='#6c757d', width=width, dash='dash'),
                          marker=dict(symbol="arrow-left", size=9)))
fig.add_traces(go.Scatter(y=data_LO_i['ScCCcomb'], mode = 'lines+markers', name="ScCCcomb", 
                          line=dict(color='black', width=width),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig, range=[0.88, 1.01])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")
fig.show()

In [167]:
data_PO_i = data_PO[data_PO['Algorithm'].isin(['GA (average)', 'ScGA', 'ACO (average)', 'ScACO', 'IWDs (average)', 
                                               'LKH', 'ScCCcomb'])][cols]
data_PO_i = data_PO_i.set_index(['Algorithm'])
data_PO_i = data_PO_i.transpose()

In [173]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=data_PO_i['GA (average)'],
                          mode = 'lines+markers', name="GA", 
                          line=dict(color='#440154', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['ScGA'],
                          mode = 'lines+markers', name="ScGA", 
                          line=dict(color='#440154', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['ACO (average)'],
                          mode = 'lines+markers', name="ACO", 
                          line=dict(color='#26828e', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_PO_i['ScACO'],
                          mode = 'lines+markers', name="ScACO", 
                          line=dict(color='#26828e', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_PO_i['IWDs (average)'],
                          mode = 'lines+markers', name="IWDs", 
                          line=dict(color='#6ece58', width=width),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['LKH'],
                          mode = 'lines+markers', name="LKH", 
                          line=dict(color='#6ece58', width=width, dash='dash'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['ScCCcomb'], mode = 'lines+markers', name="ScCCcomb", 
                          line=dict(color='#fde725', width=width),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig, range=[0.83, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")

fig.show()

In [174]:
fig = go.Figure()
width=2
fig.add_traces(go.Scatter(y=data_PO_i['GA (average)'],
                          mode = 'lines+markers', name="GA", 
                          line=dict(color='#ced4da', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['ScGA'],
                          mode = 'lines+markers', name="ScGA", 
                          line=dict(color='#ced4da', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['ACO (average)'],
                          mode = 'lines+markers', name="ACO", 
                          line=dict(color='#adb5bd', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_PO_i['ScACO'],
                          mode = 'lines+markers', name="ScACO", 
                          line=dict(color='#adb5bd', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_PO_i['IWDs (average)'],
                          mode = 'lines+markers', name="IWDs", 
                          line=dict(color='#6c757d', width=width),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['LKH'],
                          mode = 'lines+markers', name="LKH", 
                          line=dict(color='#6c757d', width=width, dash='dash'),
                          marker=dict(symbol="arrow-left", size=9)))
fig.add_traces(go.Scatter(y=data_PO_i['ScCCcomb'], mode = 'lines+markers', name="ScCCcomb", 
                          line=dict(color='black', width=width),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig, range=[0.83, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")
fig.show()

In [175]:
data_AP_i = data_AP[data_AP['Algorithm'].isin(['PSO (average)', 'ScPSO', 'RGA (average)', 'ScRGA', 'DE (average)', 
                                               'ScDE', 'ScCCreal'])][cols]
data_AP_i = data_AP_i.set_index(['Algorithm'])
data_AP_i = data_AP_i.transpose()

In [179]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=data_AP_i['PSO (average)'],
                          mode = 'lines+markers', name="PSO", 
                          line=dict(color='#440154', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['ScPSO'],
                          mode = 'lines+markers', name="ScPSO", 
                          line=dict(color='#440154', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['RGA (average)'],
                          mode = 'lines+markers', name="RGA", 
                          line=dict(color='#26828e', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_AP_i['ScRGA'],
                          mode = 'lines+markers', name="ScRGA", 
                          line=dict(color='#26828e', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_AP_i['DE (average)'],
                          mode = 'lines+markers', name="DE", 
                          line=dict(color='#6ece58', width=width),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['ScDE'],
                          mode = 'lines+markers', name="ScDE", 
                          line=dict(color='#6ece58', width=width, dash='dash'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['ScCCreal'], mode = 'lines+markers', name="ScCCreal", 
                          line=dict(color='#fde725', width=width),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig, range=[0.8, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")

fig.show()

In [180]:
fig = go.Figure()
width=2
fig.add_traces(go.Scatter(y=data_AP_i['PSO (average)'],
                          mode = 'lines+markers', name="PSO", 
                          line=dict(color='#ced4da', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['ScPSO'],
                          mode = 'lines+markers', name="ScPSO", 
                          line=dict(color='#ced4da', width=width, dash='dash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['RGA (average)'],
                          mode = 'lines+markers', name="RGA", 
                          line=dict(color='#adb5bd', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_AP_i['ScRGA'],
                          mode = 'lines+markers', name="ScRGA", 
                          line=dict(color='#adb5bd', width=width, dash='dash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_AP_i['DE (average)'],
                          mode = 'lines+markers', name="DE", 
                          line=dict(color='#6c757d', width=width),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['ScDE'],
                          mode = 'lines+markers', name="ScDE", 
                          line=dict(color='#6c757d', width=width, dash='dash'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_AP_i['ScCCreal'], mode = 'lines+markers', name="ScCCreal", 
                          line=dict(color='black', width=width),
                          marker=dict(symbol="circle", size=7)))
apply_style_scatter(fig, range=[0.8, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")
fig.show()

In [178]:
data_LO_MTP_i = data_LO_MTP[cols]
data_LO_MTP_i = data_LO_MTP_i.set_index(['Algorithm'])
data_LO_MTP_i = data_LO_MTP_i.transpose()

In [183]:
fig = go.Figure()
width=3
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScCCreal'],
                          mode = 'lines+markers', name="ScCCcomb ScCCreal", 
                          line=dict(color='#440154', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScPSOreal'],
                          mode = 'lines+markers', name="ScCCcomb ScPSOreal", 
                          line=dict(color='#440154', width=width, dash='dot'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScRGAreal'],
                          mode = 'lines+markers', name="ScCCcomb ScRGAreal", 
                          line=dict(color='#440154', width=width, dash='longdash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScDEreal'],
                          mode = 'lines+markers', name="ScCCcomb ScDEreal", 
                          line=dict(color='#440154', width=width, dash='dashdot'),
                          marker=dict(symbol="square", size=7)))

fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScCCreal'],
                          mode = 'lines+markers', name="ScACOcomb ScCCreal", 
                          line=dict(color='#26828e', width=width),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScPSOreal'],
                          mode = 'lines+markers', name="ScACOcomb ScPSOreal", 
                          line=dict(color='#26828e', width=width, dash='dot'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScRGAreal'],
                          mode = 'lines+markers', name="ScACOcomb ScRGAreal", 
                          line=dict(color='#26828e', width=width, dash='longdash'),
                          marker=dict(symbol="star", size=9)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScDEreal'],
                          mode = 'lines+markers', name="ScACOcomb ScDEreal", 
                          line=dict(color='#26828e', width=width, dash='dashdot'),
                          marker=dict(symbol="star", size=9)))

fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScCCreal'],
                          mode = 'lines+markers', name="ScGAcomb ScCCreal", 
                          line=dict(color='#6ece58', width=width),
                          marker=dict(symbol="circle", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScPSOreal'],
                          mode = 'lines+markers', name="ScGAcomb ScPSOreal", 
                          line=dict(color='#6ece58', width=width, dash='dot'),
                          marker=dict(symbol="circle", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScRGAreal'],
                          mode = 'lines+markers', name="ScGAcomb ScRGAreal", 
                          line=dict(color='#6ece58', width=width, dash='longdash'),
                          marker=dict(symbol="circle", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScDEreal'],
                          mode = 'lines+markers', name="ScGAcomb ScDEreal", 
                          line=dict(color='#6ece58', width=width, dash='dashdot'),
                          marker=dict(symbol="circle", size=7)))

apply_style_scatter(fig, width=900, height=550, range=[0.83, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")

fig.show()

In [184]:
fig = go.Figure()
width=2
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScCCreal'],
                          mode = 'lines+markers', name="ScCCcomb ScCCreal", 
                          line=dict(color='#ced4da', width=width),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScPSOreal'],
                          mode = 'lines+markers', name="ScCCcomb ScPSOreal", 
                          line=dict(color='#ced4da', width=width, dash='dot'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScRGAreal'],
                          mode = 'lines+markers', name="ScCCcomb ScRGAreal", 
                          line=dict(color='#ced4da', width=width, dash='longdash'),
                          marker=dict(symbol="x", size=9)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScCCcomb ScDEreal'],
                          mode = 'lines+markers', name="ScCCcomb ScDEreal", 
                          line=dict(color='#ced4da', width=width, dash='dashdot'),
                          marker=dict(symbol="x", size=9)))

fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScCCreal'],
                          mode = 'lines+markers', name="ScACOcomb ScCCreal", 
                          line=dict(color='#adb5bd', width=width),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScPSOreal'],
                          mode = 'lines+markers', name="ScACOcomb ScPSOreal", 
                          line=dict(color='#adb5bd', width=width, dash='dot'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScRGAreal'],
                          mode = 'lines+markers', name="ScACOcomb ScRGAreal", 
                          line=dict(color='#adb5bd', width=width, dash='longdash'),
                          marker=dict(symbol="square", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScACOcomb ScDEreal'],
                          mode = 'lines+markers', name="ScACOcomb ScDEreal", 
                          line=dict(color='#adb5bd', width=width, dash='dashdot'),
                          marker=dict(symbol="square", size=7)))

fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScCCreal'],
                          mode = 'lines+markers', name="ScGAcomb ScCCreal", 
                          line=dict(color='#6c757d', width=width),
                          marker=dict(symbol="circle", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScPSOreal'],
                          mode = 'lines+markers', name="ScGAcomb ScPSOreal", 
                          line=dict(color='#6c757d', width=width, dash='dot'),
                          marker=dict(symbol="circle", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScRGAreal'],
                          mode = 'lines+markers', name="ScGAcomb ScRGAreal", 
                          line=dict(color='#6c757d', width=width, dash='longdash'),
                          marker=dict(symbol="circle", size=7)))
fig.add_traces(go.Scatter(y=data_LO_MTP_i['ScGAcomb ScDEreal'],
                          mode = 'lines+markers', name="ScGAcomb ScDEreal", 
                          line=dict(color='#6c757d', width=width, dash='dashdot'),
                          marker=dict(symbol="circle", size=7)))

apply_style_scatter(fig, width=900, height=550, range=[0.83, 1])
fig.update_xaxes(title="Problem")
fig.update_yaxes(title="Normalized makespan")
fig.show()